In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ImportError: cannot import name 'sip' from 'PyQt5' (/home/paul/anaconda3/lib/python3.8/site-packages/PyQt5/__init__.py)

In [ ]:
df_train = pd.read_csv('input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
df_train.describe()

Now that I have got a general idea about your data set, it’s also a good idea to take a closer look at the data itself. With the help of the head() and tail() functions to check out the first and last lines of my DataFrame, respectively.

In [ ]:
df_train.head()

In [ ]:
df_train.tail()

In [ ]:
# Get the overall concise summary of the DataFrame
df_train.info()

Lets check what are the numerical features in the df_train dataset

In [ ]:
numeric_features = df_train.select_dtypes(include=[np.number])
numeric_features.columns

numeric features head

In [ ]:
numeric_features.head()

Now a general description of SalePrice

In [ ]:
# Now, as we will be predicting 'SalePrice' lets see description of that column
df_train['SalePrice'].describe()

For a regression problem, the most important thing to do is to understand the distribution
of your target. If a target ranges between 1 and 10, and after training our model we get a
mean absolute error of 5, we can tell that the error is large in this context.
However, the same error for a target that ranges between 500,000 and 1,000,000 is
negligible.

- If you have numeric type dataset and want to visualize in histogram
- then the seaborn histogram will help you. For this seaborn distplot function responsible to plot it.
- On y-axis give the numeric dataset
- On x-axis gives bins. It means distribute given dataset in a particular range and show in bars
- e.g. bins= 18 and bins = 40

In [ ]:
sns.distplot(df_train['SalePrice'])

From the above distplot, the 'SalePrice' is dense between 100k and 250k, but there are larger outliers on the right side, i.e the expensive price range.

Now, let’s have a look at the greater living area (square feet) against the sale price:

In [ ]:
# first wo quickly view all the column names in the data
# print(df_train.columns)
# for above I could also use - data.columns
# Checking - 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train['GrLivArea']], axis=1)
data.plot.scatter(x='GrLivArea', y='SalePrice', ylim=(0, 800000), s=32)

 Its generally expected that larger living area comes with higher price. This chart shows thats generally true. But there are some 2–3 “cheap” houses with large living area
Now lets explore the “TotalBsmtSF” column — Total square feet of the basement area.

In [ ]:
# Checking - 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train['TotalBsmtSF']], axis=1)
data.plot.scatter(x='TotalBsmtSF', y='SalePrice', ylim=(0, 800000))

We can see that basement area indeed may have good predictive power for the model.

Now lets check “OverallQual” — overall material and finish quality. This one may be more subjective feature.

### Now list of variables that contain year information, because I will compare the difference between All years feature with SalePrice

In [ ]:
year_feature = [feature for feature in numeric_features if 'Yr' in feature or 'Year' in feature]
year_feature

In [ ]:
# Checking - 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train['YearBuilt']], axis=1)
data.plot.scatter(x = 'YearBuilt', y = 'SalePrice', ylim=(0, 800000))

In [ ]:
# Checking - 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train['GarageYrBlt']], axis=1)
data.plot.scatter(x = 'GarageYrBlt', y = 'SalePrice', ylim=(0, 800000))

In [ ]:
# Checking - 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train['YearRemodAdd']], axis=1)
data.plot.scatter(x = 'YearRemodAdd', y = 'SalePrice', ylim=(0, 800000))

In [ ]:
# Checking - 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train['YrSold']], axis=1)
data.plot.scatter(x = 'YrSold', y = 'SalePrice', ylim=(0, 800000))


In [ ]:
# Checking - TotRmsAbvGrd' i.e. total rooms above grade
data = pd.concat([df_train['SalePrice'], df_train['TotRmsAbvGrd']], axis=1)
data.plot.scatter(x = 'TotRmsAbvGrd', y = 'SalePrice', ylim=(0, 800000))


In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['OverallQual']], axis=1)
f, ax = plt.subplots(figsize=(14, 8))
fig = sns.boxplot(x='OverallQual', y='SalePrice', data=data)
fig.axis(ymin= 0, ymax=800000)

In [ ]:

# Now lets view the top 8 correlated features with the sale price:
corr_mat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corr_mat, vmax=0.8, square=True)

In [ ]:
k = 9 # number of variables for the heatmap
cols = corr_mat.nlargest(k, 'SalePrice')['SalePrice'].index
f, ax = plt.subplots(figsize=(14,10))
sns.heatmap(df_train[cols].corr(), vmax=0.8, square=True, annot=True)

Now a simple Feature Engineering - of finding out the top-most correlated columns with SalesPrice. 
Later while in our Model will use only these top-most important features to predict SalePrices.

Feature Engineering - Find out the most correlated columns with SalesPrice and use only those columns in modelNow the big beautiful scatter plot with a hell of lot of visual data in a single plot

In [ ]:
top_correlations = df_train.corr()
top_feature_columns = top_correlations['SalePrice'][top_correlations['SalePrice'].values > 0.2].index.values
top_feature_columns

And now another heatmap with ONLY the top 10 correlated features

In [ ]:
# Handling Missing Values for 19 features which have missing values mentioned above
df_train['GarageYrBlt'] = df_train['GarageYrBlt'].fillna(0)
# filling in missing GarageYrBuilt values with zeros. But this may not be the most 
# logical approach. Refer to this discussion below.
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/discussion/60143

heat_map_with_top_correlated_features = np.append(top_feature_columns[-10:], np.array(['SalePrice']))
pearson_correlation_coefficients = np.corrcoef(df_train[heat_map_with_top_correlated_features[::-1]].T)
plt.figure(figsize=(16,16))
sns.set(font_scale=1)
with sns.axes_style('white'):
    sns.heatmap(pearson_correlation_coefficients, yticklabels=heat_map_with_top_correlated_features[::-1], xticklabels=heat_map_with_top_correlated_features[::-1], fmt='.2f', annot_kws={'size': 10}, annot=True, square=True, cmap=None)
    
    

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd']
sns.pairplot(df_train[cols], height=4)
plt.show()

Checking out missing data

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum() / df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

## Now the all important SalePrice prediction, Using Linear Regression

## Loss function

Given our Simple Linear Regression equation:

$$Y = bX + a$$

We can use the following cost function to find the coefficients:

### Mean Squared Error (MSE) Cost Function

The MSE is defined as:

$$MSE = J(W) =  \frac{1}{n} \sum_{i=1}^{n} (y^{(i)} - h_w(x^{(i)}))^2$$

where

$$h_w(x) = g(w^Tx)$$

### and Root Mean Squared Error

$$RMSE = \sqrt{(\frac{1}{n})\sum_{i=1}^{n}(y^{(i)} - h_w(x^{(i))}))^{2}}$$

Mean Squared Error (MSE) is the workhorse of basic loss functions. For calculating MSE, I take the difference between my y value predictions and the ground truth, square it, and average it out across the whole dataset.

In [ ]:
def loss_func(y_predicted, y_actual):
    squared_error = (y_predicted - y_actual) ** 2
    sum_squared_error = squared_error.sum()
    size = len(y_actual)
    return 1/(2*size) * sum_squared_error

In [ ]:
x = df_train['GrLivArea']
y = df_train['SalePrice']

x = (x - x.mean()) / x.std()
x = np.c_[np.ones(x.shape[0]), x]
x.shape

##### Now lets define our Hypothesis variable, meaning the one to be predicted, and here we are predicting 'Sale Price'. And also the independent variable. Lets say first I want to make the independent variable as 'GrLivArea' which represents, "Above grade (ground) living area square feet"

In [ ]:
x = df_train['GrLivArea']
y = df_train['SalePrice']

 I will preprocess the data using the following formula for standardization

$x′=(x−μ) / σ$

The above formulae is called Z-score is one of the most popular methods to standardize data. It is equal to the original variable, minus its mean, divided by its standard deviation.
Once the standardization is done, all the features will have a mean of zero, a standard deviation of one, and thus, the same scale.
This process produces standard scores that represent the number of standard deviations above or below the mean that a specific observation falls. For instance, a standardized value of 2 indicates that the observation falls 2 standard deviations above the mean. This interpretation is true regardless of the type of variable that you standardize.

In [ ]:
x = (x - x.mean()) / x.std()
print(x.shape)
# (1460, ) i.e. it will be an array like
# [0 1 2 3 4 5 6 7 8.... 1459]

x = np.c_[np.ones(x.shape[0]), x]
print(x.shape)
# (1460, 2)

In [ ]:
# Now below I am defining a basic Linear regression function

class SimpleLinearRegression:

    def get_predictions(self, X):
        return np.dot(X, self._W)

    def _get_gradient_descent_step(self, X, targets, learning_rate):
        predictions = self.get_predictions(X)

        error = predictions - targets
        gradient = np.dot(X.T, error)/len(x)

        # now update the W
        self._W -= learning_rate * gradient


    def fit(self, X, y, iterations_num=1000, learning_rate=0.01):
            self._W = np.zeros(X.shape[1])

            self._history_of_cost = []
            self._w_history = [self._W]

            for i in range(iterations_num):
                predictions = self.get_predictions(X)
                cost = loss_func(predictions, y)

                self._history_of_cost.append(cost)

                self._get_gradient_descent_step(x, y, learning_rate)

                self._w_history.append(self._W.copy())

            return self


In [ ]:
house_price_linear_result = SimpleLinearRegression()
house_price_linear_result.fit(x, y, iterations_num=1000, learning_rate=0.01)
print(house_price_linear_result._W)

In [ ]:
# Now plot the Cost function
plt.title('Kaggle House Price Cost Function')
plt.xlabel('No of Iterations')
plt.ylabel('House Price Cost')
plt.plot(house_price_linear_result._history_of_cost)
plt.show()
